In [6]:
try:
  import pyspark
except:
  !pip install pyspark
  import pyspark
finally:
  from pyspark.sql import SparkSession

In [7]:
sparkSession = SparkSession.builder.appName('Spark basics').getOrCreate()
sparkSession.sparkContext.setLogLevel("DEBUG")